In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import kerastuner as kt

np.random.seed(42)
tf.random.set_seed(42)


In [2]:
# lists possible devices (CPU, GPU)
# used to check if GPU is recognized/exists

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10008397188277768808
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5780226720
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15862190839086106535
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
# preprocessing of the images applied when loading image data set from disk with
# tensorflows flow_from_directory

image_gen = keras.preprocessing.image.ImageDataGenerator(rotation_range=20,  # rotate the image 20 degrees
                                                         width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255,  # Rescale the image by normalzing it.
                                                         shear_range=0.15,
                                                         # Shear means cutting away part of the image (max 20%)
                                                         zoom_range=0.15,  # Zoom in by 15% max
                                                         horizontal_flip=True,  # Allow horizontal flipping
                                                         fill_mode='nearest'
                                                         # Fill in missing pixels with the nearest filled value
                                                         )

In [4]:
# path to the image data set

#train_data_path = './drive/MyDrive/Bachelor/data_balanced/train' #google colab
#test_data_path = './drive/MyDrive/Bachelor/data_balanced/test' #google colab
#validation_data_path = './drive/MyDrive/Bachelor/data_balanced/validation' #google colab
train_data_path = './data/test-bed-deffects/data_balanced/train'  #local notebook
test_data_path = './data/test-bed-deffects/data_balanced/test'  #local notebook
validation_data_path = './data/test-bed-deffects/data_balanced/validation'  #local notebook


In [5]:
# generate training set by loading the images from their directories with flow_from_directory
# important: the folder structure has to match! i.e {train} -> {ok,def}
# at the "same time" the data augmentation is applied on the images through the ImageDataGenerator

train_image_gen = image_gen.flow_from_directory(train_data_path,
                                                target_size=(180, 180),
                                                class_mode='binary')

Found 2000 images belonging to 2 classes.


In [6]:
# generate test set by loading the images from their directories with flow_from_directory
# important: the folder structure has to match! i.e {test} -> {ok,def}
# at the "same time" the data augmentation is applied on the images through the ImageDataGenerator

test_image_gen = image_gen.flow_from_directory(test_data_path,
                                               target_size=(180, 180),
                                               class_mode='binary')

Found 998 images belonging to 2 classes.


In [7]:
# generate validation set by loading the images from their directories with flow_from_directory
# important: the folder structure has to match! i.e {validation} -> {ok,def}
# at the "same time" the data augmentation is applied on the images through the ImageDataGenerator

valid_set = image_gen.flow_from_directory(validation_data_path,
                                          target_size=(180, 180),
                                          class_mode='binary')

Found 1000 images belonging to 2 classes.


In [101]:
# saves the model and its weights as a json file in the folder saved_models

def save_model(model, model_name):
    my_model = model.to_json()
    with open(f'./saved_models/{model_name}.json', "w") as file:
        file.write(my_model)
    # serialize weights to HDF5
    model.save_weights(f'./saved_model/{model_name}_weights.h5')

In [102]:
#TODO: load model no compiling ?

def load_model(model_path, weight_path):
    # load json and create model
    file = open(model_path, 'r')
    model_json = file.read()
    file.close()
    loaded_model = keras.models.model_from_json(model_json)
    # load weights
    loaded_model.load_weights(weight_path)
    optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
    loaded_model.compile(loss="binary_crossentropy", optimizer=optimizer,
                         metrics=['accuracy', 'Recall', 'Precision', 'AUC'])
    return loaded_model

In [8]:
# method to select the base model architecture
# pretrained with the weights of the imagenet data set
# the top layers are excluded and have to be implemented on top of the base model
# currently Vgg16, Xception, ResNet50

def build_base_model(model_selected):
    if model_selected == 'vgg':
        return keras.applications.vgg16.VGG16(include_top=False, weights='imagenet')
    if model_selected == 'xception':
        return keras.applications.xception.Xception(include_top=False, weights='imagenet')
    if model_selected == 'resnet':
        return keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet')

In [9]:
# method which adds the new top layers on the base model and returns the now complete model

def model_builder():
    model_input = keras.Input(shape=(180, 180, 3), name='image_input')
    base_model = build_base_model('vgg')

    output_base_model = base_model(model_input)

    x = keras.layers.Flatten(name='flatten')(output_base_model)
    x = keras.layers.Dense(1000)(x)
    x = keras.layers.Dropout(0.25)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(1, activation='sigmoid')(x)

    return keras.models.Model(model_input, x)

In [22]:
def compile_for_tuning(lr, beta_1, beta_2):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


In [23]:
def build_model(hp):
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    beta_1 = hp.Float("beta_1", min_value=0.5, max_value=0.9, step=0.1)
    beta_2 = hp.Float("beta_2", min_value=0.80, max_value=0.99, step=0.01)
    model = compile_for_tuning(lr=lr, beta_1=beta_1, beta_2=beta_2)
    return model

In [13]:
# method activate the base models layers ability to learn (be trainable)
# set to false for the first training run to not destroy the pretrained weights

def set_layers_trainable(trainable, input_model):
    for layer in input_model.layers:
        layer.trainable = trainable

In [14]:
def compile_model(alpha, beta1, beta2, metrics):
    optimizer = keras.optimizers.Adam(learning_rate=alpha, beta_1=beta1, beta_2=beta2)
    model = model_builder()
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=metrics)

    return model

In [15]:
def train_model(input_model, train_set, test_set, epochs, callback):
    return input_model.fit(train_set,
                     validation_data=test_set,
                     epochs=epochs,
                     callbacks=callback)

In [24]:
# set base model
base_model = build_base_model('vgg')

# freeze layers of base model for first train run
set_layers_trainable(False, base_model)


# compile model
model = compile_model(0.0001, 0.9, 0.999, ['accuracy', 'Recall', 'Precision', 'AUC'])

# train frozen layer model
history = train_model(model, train_image_gen, test_image_gen, 1, [])

# unfreeze base model layers
set_layers_trainable(True, base_model)

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=3,
    overwrite=True,
    directory="test_tuner",
    project_name="test",
)

tuner.search(train_image_gen, epochs=2, validation_data=test_image_gen)

63/63 [==============================] - ETA: 1:23 - loss: 1.0165 - accuracy: 0.5312 - recall: 0.5000 - precision: 0.4667 - auc: 0.4722 - ETA: 14s - loss: 0.9365 - accuracy: 0.5625 - recall: 0.5577 - precision: 0.4833 - auc: 0.5355  - ETA: 14s - loss: 0.9248 - accuracy: 0.5625 - recall: 0.5602 - precision: 0.5028 - auc: 0.5403 - ETA: 14s - loss: 0.9249 - accuracy: 0.5625 - recall: 0.5600 - precision: 0.5080 - auc: 0.5416 - ETA: 13s - loss: 0.9249 - accuracy: 

/home/msk/anaconda3/envs/tf2/lib/python3.7/site-packages/kerastuner/engine/metrics_tracking.py:92: RuntimeWarning: All-NaN axis encountered
  return np.nanmin(values)


Epoch 1/2
63/63 [==============================] - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.4734 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.4705 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.4676 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.4653 - ETA: 11s - loss: 0.0000e+00 - accuracy: 0.4641 - ETA: 11s - loss: 0.0000e+00 - accuracy: 0.4630 - ETA: 11s - loss: 0.0000e+00 - accuracy: 0.4624

Epoch 1/2
63/63 [==============================] - ETA: 13s - loss: 0.0000e+00 - accuracy: 0.5580 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.5547 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.5513 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.5481 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.5452 - ETA: 11s - loss: 0.0000e+00 - accuracy: 0.5428 - ETA: 11s - loss: 0.0000e+00 - accuracy: 0.5402

Epoch 1/2
63/63 [==============================] - ETA: 13s - loss: 0.0000e+00 - accuracy: 0.4509 - ETA: 13s - loss: 0.0000e+00 - accuracy: 0.4507 - ETA: 13s - loss: 0.0000e+00 - accuracy: 0.4531 - ETA: 13s - loss: 0.0000e+00 - accuracy: 0.4553 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.4568 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.4584 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.4598

Epoch 1/2
63/63 [==============================] - ETA: 13s - loss: 0.0000e+00 - accuracy: 0.5761 - ETA: 13s - loss: 0.0000e+00 - accuracy: 0.5709 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.5673 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.5643 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.5608 - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.5573 - ETA: 11s - loss: 0.0000e+00 - accuracy: 0.5545

INFO:tensorflow:Oracle triggered exit


In [25]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 180, 180, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              12801000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000)              4000      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1001

In [28]:
tuner.results_summary()

In [27]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('beta_1')} and the optimal learning rate for the optimizer
is {best_hps.get('lr')}. {best_hps.get('beta_2')}
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 0.7999999999999999 and the optimal learning rate for the optimizer
is 0.006452847906140774. 0.81

